In [32]:
%reset -sf

In [33]:
import torch
from torch.utils.checkpoint import checkpoint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm
from opt_einsum import contract
from torch.profiler import profile, record_function, ProfilerActivity
def _toN(t):
    return t.detach().cpu().numpy()
def printDiff(name,value,ref):
    if ref is not None:
        print(name+':',value,'diff(abs):',value-ref)
    else:
        print(name+':',value)
    
import ast
def eval_np_array_literal(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.array(ast.literal_eval(array_string))


# SRG BaseClass

In [42]:
class SRG(torch.nn.Module):
    def __init__(self,params,options):
        super(SRG,self).__init__()
        self.dtype={'float64':torch.float64,'float32':torch.float32}[options.get('dtype','float64')]
        self.device=options.get('device','cpu')
        self.max_dim=options.get('max_dim',16)
        self.nLayers=options.get('nLayers',10)
        self.use_checkpoint=options.get('use_checkpoint',True)
        self.observable_checkerboard=False
        
        self.params=torch.nn.ParameterDict({
            k:torch.nn.Parameter(torch.tensor(v,dtype=self.dtype,device=self.device)) for k,v in params.items()
        })
        self.logZ=0
        
    def __str__(self):
        rtval=""
        for k,v in self.params.items():
            rtval+=k+':'+v+'\n'
        rtval+='dtype:'+self.dtype+'\n'
        rtval+='device:'+self.device+'\n'
        rtval+='max_dim:'+self.max_dim+'\n'
        rtval+='nLayers:'+self.nLayers+'\n'
        rtval+='nSite:'+2**nLayers+'\n'
        
    def set_params(self,params):
        self.params=torch.nn.ParameterDict({
            k:torch.nn.Parameter(torch.tensor(v,dtype=self.dtype,device=self.device)) for k,v in params.items()
        })
        
    def toT(self,t):
        return torch.tensor(t,dtype=self.dtype,device=self.device)
    
    def generate_random_Isometry(self,dim1,dim2):
        dim=max(dim1,dim2)
        A=torch.randn(dim,dim,dtype=self.dtype,device=self.device)
        U=torch.matrix_exp(A-A.t())
        U=U[:dim1,:dim2]
        return U
    
    def TRG_same_T(self,T,*w):
        return self.TRG(T,T,*w)
    
    def forward_tensor(self):
        logTotal=0
        T=self.get_T0()
        for i in range(self.nLayers):
            w=self.ws[(i*self.w_per_layer):((i+1)*self.w_per_layer)]
            if self.use_checkpoint:
                T=torch.utils.checkpoint.checkpoint(self.TRG_same_T,T,*w)
            else:
                T=self.TRG_same_T(T,*w)
                
            norm=torch.linalg.norm(T)
            T=T/norm
            logTotal=2*logTotal+torch.log(norm)
        return T,logTotal
    
    def forward_tensor_with_observable(self,T_op,contract_method=None):
        logTotal=0
        T=self.get_T0()
        contracted=torch.zeros(self.nLayers,dtype=self.dtype,device=self.device)
        for i in range(self.nLayers):
            w=self.ws[(i*self.w_per_layer):((i+1)*self.w_per_layer)]
            if self.use_checkpoint:
                T1=torch.utils.checkpoint.checkpoint(self.TRG_same_T,T,*w)
                T2=torch.utils.checkpoint.checkpoint(self.TRG,T,T_op,*w)
                T3=torch.utils.checkpoint.checkpoint(self.TRG,T_op,T,*w)
            else:
                T1=self.TRG_same_T(T,*w)
                T2=self.TRG(T,T_op,*w)
                T3=self.TRG(T_op,T,*w)
            if self.observable_checkerboard and i<self.spacial_dim:
                T3=-T3

            T,T_op=T1,(T2+T3)/2
            norm=torch.linalg.norm(T)
            T,T_op=T/norm,T_op/norm
            logTotal=2*logTotal+torch.log(norm)
            
            if contract_method is not None:
                Z=contract(T,contract_method)
                Z_op=contract(T_op,contract_method)
                contracted[i]=Z_op/Z
            
        return T,T_op,logTotal,contracted
    
    
    def dlogZ(self,param):
        self.requires_grad_(False)
        self.params[param].requires_grad_(True)
        self.zero_grad()
        logZ=self.forward()
        logZ.backward()
        result=_toN(self.params[param].grad)
        self.params[param].requires_grad_(False)
        return result
    
    def update_single_layer(self,layer):
        self.requires_grad_(False)
        
        for i in range(layer*self.w_per_layer,(layer+1)*self.w_per_layer):
            self.ws[i].requires_grad_(True)
        self.zero_grad()
        
        logZ=self.forward()
        logZ.backward()
        
        with torch.no_grad():
            for i in range(layer*self.w_per_layer,(layer+1)*self.w_per_layer):
                E=self.ws[i].grad
                dim1,dim2=E.shape[0],E.shape[2]
                E=E.reshape(dim1*dim1,dim2)
                U,S,Vh=torch.linalg.svd(E,full_matrices=False)
                UVh=U@Vh
                #UVh=svd2UVh(E)
                del U,S,Vh,E
                
                #calculate diff
                UVh_old=self.ws[i].reshape(dim1*dim1,dim2)
                self.ws_diff[i]=_toN(torch.norm(UVh_old.t()@UVh@UVh.t()@UVh_old-torch.eye(dim2,device=UVh.device)))
                del UVh_old
                    
                self.ws[i].data=UVh.reshape(dim1,dim1,dim2)
                del UVh
                torch.cuda.empty_cache()
        return _toN(logZ)
        
    def optimize(self,nIter):
        self.ws_diff=np.zeros(len(self.ws))
        
        torch.cuda.empty_cache()
        if nIter>1:
            pbar2=tqdm(range(nIter), leave=False)
            pbar2.set_postfix({k:_toN(v) for k,v in self.params.items()})
        else:
            pbar2=range(nIter)
        for i in pbar2:
            pbar=tqdm([*range(self.nLayers-1,-1,-1)]+[*range(self.nLayers)], leave=False)
            for j in pbar:
                new_logZ=self.update_single_layer(j)
        #lock all grads
        for param in self.params.values(): 
            param.requires_grad_(False)
        for i in range(self.nLayers): #slightly faster
            self.ws[i].requires_grad_(False)
        
        self.logZ_diff=np.abs(self.logZ-new_logZ)
        self.logZ=new_logZ
        # normalized by layer weight, number of elements in tensor
        # NOT USED but multiply by number of elements in last tensor to better match the effects in output
        self.ws_diff_normalized=np.zeros(len(self.ws))
        for i in range(self.nLayers):
            for j in range(self.w_per_layer):
                ij=i*self.w_per_layer+j
                self.ws_diff_normalized[ij]=self.ws_diff[ij]/2**i#/torch.numel(self.ws[ij])*torch.numel(self.ws[-1])
        # ignore the last layers we take trace directly
        # use 10-norm so layers of large error has beter contribution       
        self.ws_diff_total=np.average(self.ws_diff_normalized[:-self.w_per_layer*self.spacial_dim])
        


In [43]:
class HOTRG(SRG):
    def __init__(self,params,options):
        super(HOTRG,self).__init__(params,options)
    
    def create_isometries(self,start_dim,spacial_dim):
        ws=[]
        bond_dim=[start_dim]*spacial_dim
        for i in range(self.nLayers):
            for j in range(1,spacial_dim):
                old_dim=bond_dim[j]
                new_dim=min(old_dim**2,self.max_dim)
                U=self.generate_random_Isometry(old_dim**2,new_dim).view(old_dim,old_dim,new_dim)
                ws.append(U.detach())
                bond_dim[j]=new_dim
            bond_dim=bond_dim[1:]+[bond_dim[0]]
        self.ws=torch.nn.ParameterList([
            torch.nn.Parameter(v) for v in ws
        ])
        self.w_per_layer=spacial_dim-1
        self.spacial_dim=spacial_dim
        
    def HOTRG2D(self,T1,T2,w):
        return contract('ijkl,jmno,kna,lob->abim',T1,T2,w,w)#contract and rotate
    
    def HOTRG3D(self,T1,T2,w1,w2):
        return contract('ijklmn,jopqrs,kpa,lqb,mrc,nsd->abcdio',T1,T2,w1,w1,w2,w2)#contract and rotate
    
    def forward(self):
        T,logTotal=self.forward_tensor()
        contract_all=[i for i in range(len(T.shape)//2) for j in range(2)]
        Z=contract(T,contract_all)
        return (torch.log(Z)+logTotal)/2**self.nLayers
    
    def forward_with_observable(self,T_op):
        contract_method=[i for i in range(len(T_op.shape)//2) for j in range(2)]
        _,_,_,contracted=self.forward_tensor_with_observable(T_op,contract_method=contract_method)
        return contracted


# AKLT Public

In [44]:
from scipy.special import comb
def get_CG_no_normalization(n):
    if n==0:
        return np.eye(1)
    CG=np.zeros((n+1,)+(2,)*n)
    for i in range(2**n):
        indices=tuple(map(int,bin(i)[2:].zfill(n)))
        m=np.sum(indices)
        CG[(m,)+indices]=1
    return CG
def get_CG(n):
    if n==0:
        return np.eye(1)
    CG=np.zeros((n+1,)+(2,)*n)
    for i in range(2**n):
        indices=tuple(map(int,bin(i)[2:].zfill(n)))
        m=np.sum(indices)
        CG[(m,)+indices]=1/np.sqrt(comb(n,m))
    return CG
def get_Singlet():
    return np.array([[0,1.],[-1.,0]])


# AKLT 2D

In [45]:
class AKLT2D(HOTRG):
    default_params={'a1':np.sqrt(6)/2,'a2':np.sqrt(6)}
    def __init__(self,params,options):
        super(AKLT2D,self).__init__(params,options)
        self.create_isometries(start_dim=4,spacial_dim=2)
        self.TRG=self.HOTRG2D
        self.observable_checkerboard=True
        self.magnetization=0
        self.magnetization_diff=0
        
        
    def get_T0(self):
        projector=self.toT(get_CG_no_normalization(4))
        singlet=self.toT([[0,-1],[1,0]])
        ac0,ac1,ac2=self.toT(1),self.params['a1'],self.params['a2']
        deform=torch.stack([ac2,ac1,ac0,ac1,ac2])
        node=contract('aijkl,im,kn,a->amjnl',projector,singlet,singlet,deform)
        return contract('aijkl,amnop->imjnkolp',node,node).reshape(4,4,4,4)#UDLR

    def get_SZT0(self):
        projector=self.toT(get_CG_no_normalization(4))
        singlet=self.toT([[0,-1],[1,0]])
        ac0,ac1,ac2=self.toT(1),self.params['a1'],self.params['a2']
        deform=torch.stack([ac2,ac1,ac0,ac1,ac2])
        node=contract('aijkl,im,kn,a->amjnl',projector,singlet,singlet,deform)
        op=self.toT([2,1,0,-1,-2])
        return contract('aijkl,amnop,a->imjnkolp',node,node,op).reshape(4,4,4,4)#UDLR
    
    def calc_magnetization(self):
        with torch.no_grad():
            self.magnetization_per_layer=_toN(torch.abs(self.forward_with_observable(self.get_SZT0())))
            new_magnetization=self.magnetization_per_layer[-1]
            self.magnetization_diff=np.abs(self.magnetization-new_magnetization)
            self.magnetization=new_magnetization


Training

In [38]:
import os
import pickle
script_name=os.path.splitext(os.path.basename(__file__))[0] if '__file__' in globals() else 'notebook'

def CreateDataPoint(name,params,options):
    pickle.save(name+'.datapoint',{'params':params,'options':options})


In [39]:
def Train(name,model):
    pass

IndentationError: expected an indented block (2603096518.py, line 1)

In [46]:
options={
    'dtype':'float64',
    'device':'cuda:0',
    'max_dim':15, # 50 for float32
    'nLayers':20,
    'use_checkpoint':True
}
for a in np.linspace(1.779,1.781,5):
    params={'a1':0,'a2':a}
    datapoint_name=f'script_name[{a}]'
    CreateDataPoint(datapoint_name,params,options)

AttributeError: module 'pickle' has no attribute 'save'

In [20]:
options={
    'dtype':'float64',
    'device':'cuda:0',
    'max_dim':15, # 50 for float32
    'nLayers':20,
    'use_checkpoint':True
}
print(options)

MAX_ITER=100
EARLY_TIME_ITER=10
THRESHOLD_AVERAGE_WINDOW=10
THRESHOLD=1e-7

REUSE_W=False

sweep=(np.linspace(1.779,1.781,5))

data=pd.DataFrame()
model=AKLT2D
srg=model(model.default_params,options)

for a in sweep:
    params={'a1':0,'a2':a}
    if not REUSE_W:
        srg=model(params,options)
    srg.set_params(params)
    
    loss_curve=[]
    pbar=tqdm(range(MAX_ITER))
    
    for _iter in pbar:
        
        srg.optimize(1)
        srg.calc_magnetization()
        
        loss_curve.append(srg.ws_diff_total)
        average_loss=np.average(np.array(loss_curve[-THRESHOLD_AVERAGE_WINDOW:]))
        
        pbar.set_postfix({'ΔlogZ':srg.logZ_diff,'Δm':srg.magnetization_diff,'Δw':srg.ws_diff_total})
        
        newRow={**params,'iter':_iter+1,
                'logZ':srg.logZ,'magnetization':srg.magnetization,
                'logZ_diff':srg.logZ_diff,'magnetization_diff':srg.magnetization_diff,
                'magnetization_per_layer':srg.magnetization_per_layer.copy(),
                'ws_diff_total':srg.ws_diff_total,
                'ws_diff':srg.ws_diff.copy(),
                'ws_diff_normalized':srg.ws_diff_normalized.copy(),
                'elapsed':pbar.format_dict['elapsed'],
               }
        
        data=data.append(newRow,ignore_index=True)
        data.to_csv(script_name+'.csv')
        
        torch.save({'model':srg.state_dict(),'options':options,'params':params},script_name+f'[{a}]'+'.pth')
        saved=torch.load(script_name+f'[{a}]'+'.pth')
        #srg=model(saved['params'],saved['options'])
        #srg.load_state_dict(saved['model'])
        
        
        if _iter>=EARLY_TIME_ITER and average_loss<THRESHOLD:
            print('converged!')
            break
        
    for k,v in newRow.items():
        print(k,':',v,end=' ')
    print()


{'dtype': torch.float64, 'device': 'cuda:0', 'max_dim': 15, 'nLayers': 20, 'use_checkpoint': True}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

00:11


  0%|          | 0/40 [00:00<?, ?it/s]

00:23


  0%|          | 0/40 [00:00<?, ?it/s]

KeyboardInterrupt: 

# AKLT 3D

In [235]:
class AKLT3D(HOTRG):
    default_params={'a1':np.sqrt(20/15),'a2':np.sqrt(20/6),'a3':np.sqrt(20/1)}
    def __init__(self,params,options):
        super(AKLT3D,self).__init__(params,options)
        self.create_isometries(start_dim=4,spacial_dim=3)
        self.TRG=self.HOTRG3D
        self.observable_checkerboard=True
        self.magnetization=0
        self.magnetization_diff=0
        
    def get_T0(self):
        projector=self.toT(get_CG_no_normalization(6))
        singlet=self.toT([[0,-1],[1,0]])
        ac0,ac1,ac2,ac3=self.toT(1),self.params['a1'],self.params['a2'],self.params['a3']
        deform=torch.stack([ac3,ac2,ac1,ac0,ac1,ac2,ac3])
        node=contract('aijklmn,io,kp,mq,a->aojplqn',projector,singlet,singlet,singlet,deform)
        return contract('aijklmn,aopqrst->iojpkqlrmsnt',node,node).reshape(4,4,4,4,4,4)#UDLRFB

    def get_SZT0(self):
        projector=self.toT(get_CG_no_normalization(6))
        singlet=self.toT([[0,-1],[1,0]])
        ac0,ac1,ac2,ac3=self.toT(1),self.params['a1'],self.params['a2'],self.params['a3']
        deform=torch.stack([ac3,ac2,ac1,ac0,ac1,ac2,ac3])
        node=contract('aijklmn,io,kp,mq,a->aojplqn',projector,singlet,singlet,singlet,deform)
        op=self.toT([3,2,1,0,-1,-2,-3])
        return contract('aijklmn,aopqrst,a->iojpkqlrmsnt',node,node,op).reshape(4,4,4,4,4,4)#UDLRFB
    
    def calc_magnetization(self):
        with torch.no_grad():
            self.magnetization_per_layer=_toN(torch.abs(self.forward_with_observable(self.get_SZT0())))
            new_magnetization=self.magnetization_per_layer[-1]
            self.magnetization_diff=np.abs(self.magnetization-new_magnetization)
            self.magnetization=new_magnetization


# Determine Convergence

In [ ]:
options={
    'dtype':'float64',
    'device':'cuda:0',
    'max_dim':8, # 12 for float32 # 10 discussed with wei
    'nLayers':21, # 30
    'use_checkpoint':True
}
print(options)

MAX_ITER=100
EARLY_TIME_ITER=10
THRESHOLD_AVERAGE_WINDOW=10
THRESHOLD=1e-7

REUSE_W=False

sweep=[1,2,3,4]

data=pd.DataFrame()
model=AKLT3D
srg=model(model.default_params,options)

for a in sweep:
    params={'a1':0,'a2':0,'a3':a}
    if not REUSE_W:
        srg=model(params,options)
    srg.set_params(params)
    
    loss_curve=[]
    pbar=tqdm(range(MAX_ITER))
    
    for _iter in pbar:
        
        srg.optimize(1)
        srg.calc_magnetization()
        
        loss_curve.append(srg.ws_diff_total)
        average_loss=np.average(np.array(loss_curve[-THRESHOLD_AVERAGE_WINDOW:]))
        
        pbar.set_postfix({'ΔlogZ':srg.logZ_diff,'Δm':srg.magnetization_diff,'Δw':srg.ws_diff_total})
        
        newRow={**params,'iter':_iter+1,
                'logZ':srg.logZ,'magnetization':srg.magnetization,
                'logZ_diff':srg.logZ_diff,'magnetization_diff':srg.magnetization_diff,
                'magnetization_per_layer':srg.magnetization_per_layer.copy(),
                'ws_diff_total':srg.ws_diff_total,
                'ws_diff':srg.ws_diff.copy(),
                'ws_diff_normalized':srg.ws_diff_normalized.copy(),
                'elapsed':pbar.format_dict['elapsed'],
               }
        data=data.append(newRow,ignore_index=True)
        data.to_csv(script_name+'.csv')
        
        torch.save({'model':srg.state_dict(),'options':options,'params':params},script_name+f'[{a}]'+'.pth')
        saved=torch.load(script_name+f'[{a}]'+'.pth')
        #srg=model(saved['params'],saved['options'])
        #srg.load_state_dict(saved['model'])
        
        if _iter>=EARLY_TIME_ITER and average_loss<THRESHOLD:
            print('converged!')
            break
        
    for k,v in newRow.items():
        print(k,':',v,end=' ')
    print()


{'dtype': torch.float64, 'device': 'cuda:0', 'max_dim': 8, 'nLayers': 21, 'use_checkpoint': True}


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]